## Imports

In [ ]:
import cv2 as cv
import imageio
import matplotlib.pyplot as plt
import numpy as np
import os

from MyPython import MyCV, MyPickle, CS383
import MyData
import MyML

## Data Fetching

In [ ]:
# Import Data
X, Mean, Std = MyData.getNumbers()

## Utility Functions

In [ ]:
# Utils
def verify(img): assert (img.shape in [(784,), (1,784)]), f"should only get a single image, not {img.shape = }"
def reshape(img): return img.reshape(28,28)
def scale(img): return cv.resize(img, dsize=(280,280), interpolation=cv.INTER_NEAREST)

# Image Display and Insight
def show_img(img):
    img = scale(reshape(img))
    MyCV.show_img(img)
def show_imgs(imgs): 
    for img in imgs: show_img(img)
def img_summary(img):
    verify(img)
    return {'mean':np.average(img), 'std':np.std(img), 'min':np.min(img), 'max':np.max(img)}

# Video Writing
DEFAULT_fps = 4
DEFAULT_mbs = 14
def save_video(filename, imgs, fps=DEFAULT_fps, mbs=DEFAULT_mbs):
    imageio.mimwrite(filename, imgs, fps=fps, macro_block_size=mbs)

## Model Training (Simple)

In [ ]:
for num in range(10):

    model = MyML.GenerativeAdversarialModel(
        Data = X[num],
        DataMean = Mean[num],
        DataStd = Std[num],
        no_std = True,
        Bounds = [0,255],
        GeneratorLayers = [
            MyML.FullyConnectedLayer(784, 784),
            MyML.LinearLayer()
        ],
        DescriminatorLayers = [
            MyML.FullyConnectedLayer(784, 1),
            MyML.SigmoidLayer()
        ]
    )
    imgs = model.train(100, 10 ** -4, 10)

    save_video(f"videos/{num}.mp4", (scale(reshape(img)) for img in imgs), fps=10)


## Model Training (Extra Layer Each)

In [ ]:
for num in range(10):

    model = MyML.GenerativeAdversarialModel(
        Data = X[num],
        DataMean = Mean[num],
        DataStd = Std[num],
        no_std = True,
        Bounds = [0,255],
        GeneratorLayers = [
            MyML.FullyConnectedLayer(784, 100),
            MyML.LinearLayer(),
            MyML.FullyConnectedLayer(100,784),
            MyML.LinearLayer()
        ],
        DescriminatorLayers = [
            MyML.FullyConnectedLayer(784,100),
            MyML.LinearLayer(),
            MyML.FullyConnectedLayer(100, 1),
            MyML.SigmoidLayer()
        ]
    )
    imgs = model.train(100, 10 ** -8, 10)

    save_video(f"videos/{num}-extralayer.mp4", (scale(reshape(img)) for img in imgs), fps=10)
